<a href="https://colab.research.google.com/github/lmcanavals/machine_learning/blob/main/1302_off_to_png_and_stl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python -m pip install -U kaleido

In [11]:
import plotly.graph_objects as go
import numpy as np

In [9]:
def readOff(filename):
    f = open(filename)
    f.readline()
    nvertices, nfaces, nedges = map(int, f.readline().split())
    #print(f"Vertices: {nvertices}, Faces: {nfaces}, Edges: {nedges}")
    vertices = []
    for _ in range(nvertices):
        vertices.append(list(map(float, f.readline().strip().split())))

    vertices = np.array(vertices)

    triangles = []
    for _ in range(nfaces):
        face = list(map(int, f.readline().strip().split()))
        ntriangles, verts = face[0] - 3 + 1, face[1:]
        for n in range(ntriangles):
            triangles.append([verts[0], verts[1 + n], verts[2 + n]])

    triangles = np.array(triangles)

    return vertices, triangles

In [19]:

def save(filename):
    vertices, faces = readOff(filename)

    x, y, z = vertices.T
    I, J, K = faces.T

    mesh = go.Mesh3d(
                x=-x,
                y=y,
                z=z,
                #vertexcolor=vertices[:, 3:], #the color codes must be triplets of floats  in [0,1]!!
                i=I,
                j=J,
                k=K,
                name='',
                showscale=False,
                #opacity = 0.75,
                color = 'brown')# Download data set from plotly repo


    fig = go.Figure(data=mesh)
    fig.update_layout(
        scene = dict(
            xaxis = dict(visible=False),
            yaxis = dict(visible=False),
            zaxis =dict(visible=False)
            )
        )
    fig.write_image("fig1.png")
    #fig.show()

In [21]:
%timeit save('sofa.off')

1.44 s ± 256 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
def saveStl(vertices, triangles, name):
    f = open(name, 'w')
    f.write(f"solid {name}")

    for t in triangles:
        x0, y0, z0 = vertices[t[0]]
        x1, y1, z1 = vertices[t[1]]
        x2, y2, z2 = vertices[t[2]]
        f.write(f"""
facet normal 1 1 1
    outer loop
        vertex {x0} {y0} {z0}
        vertex {x1} {y1} {z1}
        vertex {x2} {y2} {z2}
    endloop
endfacet""")

    f.write(f"\nendsolid {name}")
    f.close()

vertices, triangles = readOff('sofa.off')
saveStl(vertices, triangles, 'test.stl')